Common coupling occurs when two or more modules share global data, leading to a situation where changes to the global data can affect all modules that use it. This type of coupling can make the code more difficult to maintain and debug because any module can modify the shared data, creating unexpected side effects in other modules.

## Example of Common Coupling:

In [39]:
# shared_data.py
global_config = {
    'app_mode': 'development',
    'max_connections': 10,
    'log_level': 'info'
}


In [40]:
# module_a.py
# from shared_data import global_config

def modify_config(new_mode, new_log_level):
    global_config['app_mode'] = new_mode
    global_config['log_level'] = new_log_level
    print(f"Module A modified app_mode to {new_mode} and log_level to {new_log_level}")


In [41]:
# module_b.py
# from shared_data import global_config

def read_config():
    print(f"Module B reads config: app_mode={global_config['app_mode']}, log_level={global_config['log_level']}")


In [42]:
# main.py
# from module_a import modify_config
# from module_b import read_config

# Initial read of the config in Module B
read_config()  # Output: Module B reads config: app_mode=development, log_level=info

# Modify the config in Module A
modify_config(new_mode='production', new_log_level='debug')

# Read the updated config in Module B
read_config()  # Output: Module B reads config: app_mode=production, log_level=debug


Module B reads config: app_mode=development, log_level=info
Module A modified app_mode to production and log_level to debug
Module B reads config: app_mode=production, log_level=debug


## Refactoring Issue


Let's modify the common coupled example to illustrate why it could be problematic. Suppose we need to add new keys or change the structure of the global_config dictionary. I'll show how this impacts the modules and creates challenges.

### Modified shared_data.py:
Suppose we decide to change global_config to include a nested structure for logging information, like this:

In [43]:
# shared_data.py
global_config = {
    'app_mode': 'development',
    'max_connections': 10,
    'logging': {
        'level': 'info',
        'file_path': '/var/log/app.log'
    }
}


In [44]:
# module_a.py (Original)
# from shared_data import global_config

def modify_config(new_mode, new_log_level):
    # This will fail or cause unintended issues
    global_config['app_mode'] = new_mode
    global_config['log_level'] = new_log_level  # This key no longer exists and will create a new key
    print(f"Module A modified app_mode to {new_mode} and log_level to {new_log_level}")

# module_b.py (Original)
# from shared_data import global_config

def read_config():
    # This will either fail or not display updated logging details correctly
    print(f"Module B reads config: app_mode={global_config['app_mode']}, log_level={global_config['log_level']}")
    # KeyError because 'log_level' was moved to the nested 'logging' dictionary


In [45]:
# Initial read of the config in Module B
try:
    print("Initial config read:")
    read_config()  # Output: Module B reads config: app_mode=development, log_level=info

    # Modify the config in Module A
    print("\nModifying config in Module A...")
    modify_config(new_mode='production', new_log_level='debug')

    # Read the updated config in Module B
    print("\nUpdated config read:")
    read_config()  # Output: Module B reads config: app_mode=production, log_level=debug
except Exception as e:
    print(e)

Initial config read:
'log_level'




### Impact of the Change:
1. **Key Errors**: `module_b.py` will raise a `KeyError` when trying to access `global_config['log_level']`, as this key no longer exists at the top level. It has been moved to `global_config['logging']['level']`.
2. **Data Inconsistencies**: `module_a.py` updates `global_config['log_level']`, which now creates a new key at the top level, leading to inconsistencies in how the configuration is represented and read across modules.
3. **Widespread Changes Needed**: To accommodate the new structure, every module that accesses `global_config` must be updated to use the new nested structure (`global_config['logging']['level']`).
4. **Error-Prone Refactoring**: Finding and updating all instances where `global_config` is used is error-prone and time-consuming, increasing the chance of missing references and introducing bugs.

### Refactored Version to Handle Changes:
To handle these changes more gracefully, refactor the code to use a `ConfigManager` class:


In [46]:
class ConfigManager:
    def __init__(self):
        self._config = {
            'app_mode': 'development',
            'max_connections': 10,
            'logging': {
                'level': 'info',
                'file_path': '/var/log/app.log'
            }
        }

    def get_config(self):
        return self._config

    def get_value(self, *keys):
        """Safely get a nested value from the configuration."""
        config_section = self._config
        for key in keys:
            if isinstance(config_section, dict) and key in config_section:
                config_section = config_section[key]
            else:
                raise KeyError(f"Configuration key {' -> '.join(keys)} not found.")
        return config_section

    def set_value(self, value, *keys):
        """Safely set a nested value in the configuration."""
        config_section = self._config
        for key in keys[:-1]:
            if key not in config_section or not isinstance(config_section[key], dict):
                config_section[key] = {}
            config_section = config_section[key]
        config_section[keys[-1]] = value

    def get_logging_level(self):
        return self.get_value('logging', 'level')

    def set_logging_level(self, level):
        self.set_value(level, 'logging', 'level')

    def set_app_mode(self, app_mode):
        self._config['app_mode'] = app_mode

    def get_app_mode(self):
        return self._config['app_mode']


# module_a.py (Refactored)
def modify_config(config_manager, new_mode, new_log_level):
    config_manager.set_app_mode(new_mode)  # Use specific method for setting app mode
    config_manager.set_logging_level(new_log_level)  # Use specific method for setting logging level
    print(f"Module A modified app_mode to {new_mode} and log_level to {new_log_level}")

# module_b.py (Refactored)
def read_config(config_manager):
    app_mode = config_manager.get_app_mode()  # Use specific method for reading app mode
    log_level = config_manager.get_logging_level()  # Use specific method for reading logging level
    print(f"Module B reads config: app_mode={app_mode}, log_level={log_level}")



In [47]:
# Initial read of the config in Module B
try:
    config = ConfigManager()
    print("Initial config read:")
    read_config(config)  # Output: Module B reads config: app_mode=development, log_level=info

    # Modify the config in Module A
    print("\nModifying config in Module A...")
    modify_config(config_manager=config, new_mode='production', new_log_level='debug')

    # Read the updated config in Module B
    print("\nUpdated config read:")
    read_config(config)  # Output: Module B reads config: app_mode=production, log_level=debug
except Exception as e:
    print(e)

Initial config read:
Module B reads config: app_mode=development, log_level=info

Modifying config in Module A...
Module A modified app_mode to production and log_level to debug

Updated config read:
Module B reads config: app_mode=production, log_level=debug
